In [1]:
import rdflib
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

# Load Word Embeddings

In [2]:
import logging
from six import iteritems
from web.embeddings import load_embedding, fetch_conceptnet_numberbatch
from web.evaluate import evaluate_similarity, evaluate_similarity_score

In [3]:
%%time
#Fetch FastText-wiki 300d 1M
we =load_embedding("C:\\Users\\daix8\\web_data\\embeddings\\fastText\\wiki-news-300d-1M.vec\\wiki-news-300d-1M.vec", format='word2vec', normalize=True, lower=False, clean_words=False)

Wall time: 3min 49s


In [41]:
%%time
#Fetch ConceptNetNumb
wecn = fetch_conceptnet_numberbatch()

File already downloaded, skipping
Wall time: 1min 37s


In [4]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# isEntityInDatabase Method

In [5]:
def isEntityInDatabase(s):
    return s.startswith("http")

# MakeQueryString Method

In [6]:
def makeQueryString(obj, filterStringsPredicates, filterStringSubjects, literals):
    if obj in literals:
        result = "SELECT distinct ?s ?p WHERE { " + "{?s ?p " + obj + " . " + "} FILTER ("
        for s in filterStringsPredicates:
            fsp = "!regex(str(?p), '" + s + "' , 'i') && "
            result += fsp
        for s in filterStringSubjects:
            fss = "!regex(str(?s), '" + s + "' , 'i') && "
            result += fss
        result = result[:result.rindex("&&")] + ")}"
    else:
        result = "SELECT distinct ?s ?p ?j ?k WHERE { "
        incomingQuery = "{?s ?p <" + obj + "> . FILTER ("
        outgoingQuery =  "{<" + obj + "> ?j ?k. FILTER ("
        
        for s in filterStringsPredicates:
            fsp = "!regex(str(?p), '" + s + "' , 'i') && "
            incomingQuery += fsp
            fsj = "!regex(str(?j), '" + s + "' , 'i') && "
            outgoingQuery += fsj
            
        for s in filterStringSubjects:
            fss = "!regex(str(?s), '" + s + "' , 'i') && "
            fss2 = "!regex(str(?k), '" + s + "' , 'i') && "
            incomingQuery += fss
            outgoingQuery += fss2
        
        incomingQuery = incomingQuery[:incomingQuery.rindex("&&")] + ")}"
        outgoingQuery = outgoingQuery[:outgoingQuery.rindex("&&")] + ")}"
        result = result + incomingQuery + " UNION " + outgoingQuery + "}"
    return result

# Triple Class 

In [7]:
class Triple:
    def __init__(self):
        self.subject = None
        self.object = None
        self.predicate = None
        self.cost = None
        self.previousTriple = None
        #the list of connecting seeds
        self.seeds = []
    
    def __str__(self):
        if(self.subject is not None and self.predicate is not None and self.object is not None):
            # print connecting seeds
            connectSeeds = "["
            for seed in self.seeds:
                connectSeeds += seed + ", "
            connectSeeds = connectSeeds[:connectSeeds.rindex(',')] + "]"
            return self.subject + "  --  " + self.predicate + "  --  " + self.object + "  " + connectSeeds + " (" + str(self.cost) + ')'
        else:
            return "Not Well Defined Triple"
        
    def __eq__(self,other):
        if(self.subject == other.getSubject() and self.object == other.getObject() and self.predicate == other.getPredicate()):
            return True
        else:
            return False
        
    def __gt__(self,other):
        if self.cost > other.getCost():
            return True
        else:
            return False
    
    def setSeeds(self, fs):
        if fs not in self.seeds:
            self.seeds.append(fs)
            self.seeds.sort()
            
    def getSeeds(self):
        return self.seeds
    
    def setPreviousTriple(self, pt):
        self.previousTriple = pt
        
    def getPreviousTriple(self):
        return self.previousTriple
    
    def updateSeeds(self, newSeeds):
        for seed in newSeeds:
            self.setSeeds(seed)
            
    def setSubject(self, sub):
        self.subject = sub
    def setObject(self, obj):
        self.object = obj
    def setPredicate(self, pre):
        self.predicate = pre
    def setCost(self, c):
        self.cost = c
    def getCost(self):
        return self.cost
    def getSubject(self):
        return self.subject
    def getObject(self):
        return self.object
    def getPredicate(self):
        return self.predicate

# Expand Method

In [8]:
'''
def expand(queryTriples, eg, di, predicatesToMatch, wemb):
    newQueryTriples = []
    matchingTriples = []
    
    tmp = []
    print("queryTriple size : " + str(len(queryTriples)))
    for queryObject in queryTriples:
        queryString = makeQueryString(queryObject.getObject(), filterStringPredicates,filterStringSubjects, literalsToConnect)
        print("----- Current Query ------")
        print(queryString)
        print("--------------------------")
        
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            if 's' in result:
                if(isEntityInDatabase(result["s"]["value"])):
                    #print(result["s"]["value"] + "   ---   " + result["p"]["value"])
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["p"]["value"])
                    newTriple.setObject(result["s"]["value"])
                    #newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["p"]["value"], wemb ))
                    #newTriple.setCost(20)
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    #print(newTriple)
                    
                    #Apply word embedding for costs and modify matchingTriples
                    if newTriple not in matchingTriples:
                        matchingTriples.append(newTriple)
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                    
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
            else:
                if(isEntityInDatabase(result["k"]["value"])):
                    #print(result["k"]["value"] + "   ---   " + result["j"]["value"])
                    
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["j"]["value"])                
                    newTriple.setObject(result["k"]["value"])
                    #newTriple.setCost(20)
                    #newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["j"]["value"], wemb ))
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    
                    #print(newTriple)
                    
                    #Apply word embedding for costs and modify matchingTriples
                    if newTriple not in matchingTriples:
                        matchingTriples.append(newTriple)
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                        
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
        computeCosts(predicatesToMatch, matchingTriples, wemb)
    return newQueryTriples, matchingTriples
'''
print("Old version of expand")

Old version of expand


In [9]:
def expand(queryTriples, eg, di, predicatesToMatch, wemb):
    newQueryTriples = []
    matchingTriples = []
    
    tmp = []
    print("queryTriple size : " + str(len(queryTriples)))
    for queryObject in queryTriples:
        queryString = makeQueryString(queryObject.getObject(), filterStringPredicates,filterStringSubjects, literalsToConnect)
        print("----- Current Query ------")
        print(queryString)
        print("--------------------------")
        
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            
            if 's' in result:
                newTriple = Triple()
                newTriple.setSubject(queryObject.getObject())
                newTriple.setPredicate(result["p"]["value"])
                newTriple.setObject(result["s"]["value"])
                newTriple.setPreviousTriple(queryObject)
                    
                for seed in queryObject.getSeeds():
                    newTriple.setSeeds(seed)
                    
                if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                
                #Apply word embedding for costs and modify matchingTriples
                if newTriple not in matchingTriples:
                    matchingTriples.append(newTriple)                       
                
                if(isEntityInDatabase(result["s"]["value"])):                                   
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                    
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
                        
            else:
                                    
                newTriple = Triple()
                newTriple.setSubject(queryObject.getObject())
                newTriple.setPredicate(result["j"]["value"])                
                newTriple.setObject(result["k"]["value"])
                newTriple.setPreviousTriple(queryObject)

                for seed in queryObject.getSeeds():
                    newTriple.setSeeds(seed)

                #Apply word embedding for costs and modify matchingTriples
                if newTriple not in matchingTriples:
                    matchingTriples.append(newTriple)

                if newTriple not in expandedGraph:
                    expandedGraph.append(newTriple)
                    
                if(isEntityInDatabase(result["k"]["value"])):
                    #print(result["k"]["value"] + "   ---   " + result["j"]["value"])

                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                        
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        #print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
        computeCosts(predicatesToMatch, matchingTriples, wemb)
    return newQueryTriples, matchingTriples

# addDuplicatedItems Method

In [10]:
def addDuplicatedItems(ntp, eg, di):
    for tp in eg:
        if tp.getObject() == ntp.getObject() and tp not in di:
            di.append(tp)
            if tp.getSeeds() != ntp.getSeeds():
                updateSeedsInExpandedGraph(tp,ntp,eg)           
    #di.append(ntp)
    return "Dup"

In [11]:
def updateSeedsInExpandedGraph(tp,ntp,eg):
    old1 = tp.getSeeds()[:]
    old2 = ntp.getSeeds()[:]
    ntp.updateSeeds(tp.getSeeds())
    newS = ntp.getSeeds()
    
    for triple in eg:
        if triple.getSeeds() == old1 or triple.getSeeds() == old2:
            triple.updateSeeds(newS)

# Reduction Tests

In [12]:
def reductionTestsDegreeOne(eg):
    
    degreeOneNodes = []
    
    for tp1 in eg:
        duplicatedObject = False
        isLeafNode = True
        for tp2 in eg:
            if tp2.getPreviousTriple() == tp1:
                isLeafNode = False
            if tp1.getObject() == tp2.getObject() and not tp1.getSubject() == tp2.getSubject():
                duplicatedObject = True
                break
        if not duplicatedObject and isLeafNode:
            degreeOneNodes.append(tp1)
    print("Degree One Nodes size: " + str(len(degreeOneNodes)))
    for tp in degreeOneNodes:
        eg.remove(tp)

#TODO: Fix Bug
def keepMinEdge(eg):
    
    triplesToBeRemoved = []
    
    for tp1 in eg:
        minTriple = tp1
        for tp2 in eg:
            if tp1.getSubject() == tp2.getSubject() and tp1.getObject() == tp2.getObject and not tp1.getPredicate() == tp2.getPredicate():
                if tp2.getCost() < tp1.getCost():
                    minTriple = tp2
                    if tp1 not in triplesToBeRemoved:
                        triplesToBeRemoved.append(tp1)
                else:
                    if tp2 not in triplesToBeRemoved:
                        triplesToBeRemoved.append(tp2)
    for tp in triplesToBeRemoved:
        eg.remove(tp)
    

# checkConnection Method

In [13]:
def checkConnection(ltc, di):
    if len(ltc) == 1:
        return False
    else:
        ltc.sort()
        for tp in di:
            if tp.getSeeds() == ltc:
                return True
        return False

# ComputeCosts Method

In [14]:
def computeCosts(predicatesToMatch, matchingTriples, wemb):
    th = 30
    predicateList = []
    
    for tp in matchingTriples:
        predicate = tp.getPredicate()
        predicateList.append(predicate[predicate.rindex('/')+1:])
        
    for y in predicatesToMatch:
        ar = []
        for p in predicateList:
            ar.append([y, p])
        ar = np.array(ar)
        result = evaluate_similarity_score(wemb, ar, 'xxx')
        #print("size of result :" + str(len(result)) + '  ----  size of matching Triples : ' + str(len(matchingTriples)))
        for x in range(len(result)):
            #when two comparing predicates are both not in the word embedding, then using JW distance instead
            #TODO: What if one of them is in word embedding?
            if result[x] == 1.0 and y != predicateList[x]:
                #print(y + "  ---  " + predicateList[x])
                result[x] = dw(y, predicateList[x])
            elif (1 - result[x]) * 100 > th:
                result[x] = dw(y, predicateList[x])
                
            if matchingTriples[x].getCost() is None:
                matchingTriples[x].setCost((1 - result[x]) * 100)
            elif matchingTriples[x].getCost() > (1 - result[x]) * 100:
                matchingTriples[x].setCost((1 - result[x]) * 100)
    
    index = 0
    
    while index < len(matchingTriples):
        if matchingTriples[index].getCost() > th:
            matchingTriples.pop(index)
        else:
            index += 1


In [15]:
def computeCosts2(predicatesToMatch, matchingTriples, wemb):
    th = 30
    predicateList = []
    
    for tp in matchingTriples:
        predicate = tp.getPredicate()
        predicateList.append(predicate[predicate.rindex('/')+1:])
        
    for y in predicatesToMatch:
        ar = []
        for p in predicateList:
            ar.append([y, p])
        ar = np.array(ar)
        result = evaluate_similarity_score(wemb, ar, 'xxx')
        #print("size of result :" + str(len(result)) + '  ----  size of matching Triples : ' + str(len(matchingTriples)))
        for x in range(len(result)):
            #when two comparing predicates are both not in the word embedding, then using JW distance instead
            #TODO: What if one of them is in word embedding?
            if result[x] == 1.0 and y != predicateList[x]:
                #print(y + "  ---  " + predicateList[x])
                result[x] = dw(y, predicateList[x])
            elif (1 - result[x]) * 100 > th:
                result[x] = dw(y, predicateList[x])
                
            if matchingTriples[x].getCost() is None:
                matchingTriples[x].setCost((1 - result[x]) * 100)
            elif matchingTriples[x].getCost() > (1 - result[x]) * 100:
                matchingTriples[x].setCost((1 - result[x]) * 100)
    
    index = 0
    
    while index < len(matchingTriples):
        if matchingTriples[index].getCost() > th:
            matchingTriples.pop(index)
        else:
            index += 1

# Identify Entities and predicates in keywords
###### 1. check if the lowercased literal is a property in dataset. If yes, then it is a predicate.
###### 2. check if the convertFirstToCapital literal is a property in dataset. If yes, then it is a predicate.
###### 3. check if the convertFirstToCapital literal is a resource in dataset. If yes, then it is an entity
###### 4. Treat every literal as a predicate.

In [26]:
def isResourceInDataset2(literal):
    jw_th = 0.7
    
    queryString = 'select distinct ?s where {?s ?p "'+ literal.lower() + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        if 'property' in sub:
            if dw(sub[sub.rindex('/')+1:],literal) > jw_th:
                return False
    
    literal = convertFirstToCapital(literal)

    queryString = 'select distinct ?s where {?s ?p "'+ literal + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    propertyInSub = False
    resourceInSub = False
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        try:
            p = dw(sub[sub.rindex('/') +1:],literal)
        except:
            print("Substring not found, treat as a property")
            return False
        
        if 'property' in sub:
            if  p > jw_th:
                propertyInSub = True
        elif 'resource' in sub:
            if p > jw_th:
                resourceInSub = True
                
    if propertyInSub:
        return False
    elif resourceInSub:
        return True
    else:
        return False

# Test version

In [49]:
def isResourceInDataset(literal):
    jw_th = 0.7
    
    #Original keyword:
    queryString = 'select distinct ?s where {?s ?p "'+ literal + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        #print(sub)
        try:
            p = dw(sub[sub.rindex('/') +1:],literal)
        except:
            print("Find a record with no / in lowercased")
            
        if 'resource' in sub and p > jw_th:
            return True
        
    #Lowercase:
    queryString = 'select distinct ?s where {?s ?p "'+ literal.lower() + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        #print(sub)
        try:
            p = dw(sub[sub.rindex('/') +1:],literal)
        except:
            print("Find a record with no / in lowercased")
            
        if 'resource' in sub and p > jw_th:
            return True
    
    #First letter capitalized
    literal = convertFirstToCapital(literal)
    
    queryString = 'select distinct ?s where {?s ?p "'+ literal + '"@en}'
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        #print(result["s"]["value"])
        sub = result["s"]["value"]
        #print(sub+ " cftc")
        try:
            p = dw(sub[sub.rindex('/') +1:],literal)
        except:
            print("Find a record with no / in CFTC")
        
        if 'resource' in sub and p > jw_th:
            return True
                
    return False

In [46]:
def indentifyEntitiesAndPredicates(keywords):
    entity = []
    pre = []
    
    li = keywords.split(',')
    
    for x in range(len(li)):
        li[x] = li[x].strip()
        #print("current key word: " + li[x])
        if isResourceInDataset(li[x]):
            entity.append(convertFirstToCapital(li[x]))
        # we treat every keyword as a predicate
        pre.append(li[x])
    
    for x in range(len(entity)):
        entity[x] = '"' + entity[x] + '"@en'
    return entity,pre


# Jaro_Winkler Distance

In [18]:
def make1shorter(word1, word2):
    if len(word1) > len(word2):
        # Make sure word1 is shorter
        temp = word1
        word1 = word2
        word2 = temp

    return word1, word2

def dj(word1, word2):
    if len(word1) == 0 or len(word2) == 0:
        raise Exception("Not words, mate")

    word1, word2 = make1shorter(word1, word2)

    word2chars = list(word2)
    m = 0
    for char in word1:
        if char in word2chars:
            m += 1
            word2chars.pop(word2chars.index(char))

    t = 0
    for i in range(len(word1)):
        if word1[i] != word2[i]:
            t += 1

    if m == 0:
        return 0
    return 1/3*(m/len(word1) + m/len(word2) + (m - t / 2)/m)

def dw(word1, word2, p=0.1, lmax=4):
    word1, word2 = make1shorter(word1, word2)

    dj_ = dj(word1, word2)

    l = 0
    for i in range(min(len(word1), lmax)):
        if word1[i] == word2[i]:
            l += 1
        else:
            break

    return dj_ + l * p * (1 - dj_)

# Convert first letter in literals to capital case and lower case for the rest

In [19]:
def convertFirstToCapital(s):
    res = s.split()
    for x in range(len(res)):
        res[x] = res[x][0].upper() + res[x][1:].lower()
    res = " ".join(res)
    return res

# Main Program

In [57]:
%%time

with open('qald5-3.txt','r',encoding='utf-8') as f:
    lines = f.readlines()

with open('test_res-3-2.txt','w',encoding='utf-8') as fw:
    for line in lines:
        filterStringPredicates = ["wikiPageWikiLink","wikiPageRedirects","wikiPageDisambiguates", "Thing","wikiPageUsesTemplate","rdf-syntax-ns#type"]
        filterStringSubjects = ["entity", "Category", "wikidata","owl#Thing", "http://wikidata.dbpedia.org/resource/Q"]
        expandedGraph = []
        duplicatedItems = []

        literalsToConnect, predicates = indentifyEntitiesAndPredicates(line.strip())
        fw.write("-----Question------\n")
        fw.write(line + '\n')
        fw.write("-------------------\n")
        qtps = []
        for literal in literalsToConnect:
            triple = Triple()
            triple.setObject(literal)
            triple.setSeeds(literal)
            qtps.append(triple)

        fw.write('--entities--\n')
        print('---- entities ----')
        for tp in qtps:
            fw.write(str(tp.getObject()) + '\n')
            print(tp.getObject())
        print(predicates)
        # Test Purpose
        #"""
        levelOfExpansion = 1
        while(levelOfExpansion < 3 and not checkConnection(literalsToConnect, duplicatedItems)):
            #add mtps to a new list
            qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
            levelOfExpansion += 1

        print("Matching Triples: -----   size: " + str(len(mtps)))
        mtps.sort()
        for tp in mtps:
            fw.write(str(tp) + '\n')
            print(tp)
        if checkConnection(literalsToConnect, duplicatedItems):
            reductionTestsDegreeOne(expandedGraph)
            fw.write("Size of expanded graph: "+ str(len(expandedGraph)) + '\n')
            for tp in expandedGraph:
                fw.write(str(tp) + '\n')
        
        #for x in expandedGraph:
        #    print(x)
#                """

Find a record with no / in lowercased
Find a record with no / in lowercased
---- entities ----
"World Of Warcraft"@en
"Develop"@en
['World of Warcraft', 'develop']
queryTriple size : 2
----- Current Query ------
SELECT distinct ?s ?p WHERE { {?s ?p "World Of Warcraft"@en . } FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )}
--------------------------


Missing 2 words. Will replace them with mean vector
Missing 1 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p WHERE { {?s ?p "Develop"@en . } FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )}
--------------------------


Missing 4 words. Will replace them with mean vector
Missing 1 words. Will replace them with mean vector


queryTriple size : 4
----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/World_Of_Warcraft> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/World_Of_Warcraft> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' 

...

----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Develop_(magazine)> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Develop_(magazine)> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(st

Missing 62 words. Will replace them with mean vector
Missing 23 words. Will replace them with mean vector


----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/DVLP> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/DVLP> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(str(?k), 'Category' , 'i') && 

Missing 102 words. Will replace them with mean vector
Missing 27 words. Will replace them with mean vector


Matching Triples: -----   size: 0
Wall time: 4.64 s


# Test Above

In [ ]:
%%time
levelOfExpansion = 1
while(levelOfExpansion < 3 or not checkConnection(literalsToConnect, duplicatedItems)):
    #add mtps to a new list
    qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
    levelOfExpansion += 1

print("Matching Triples: -----   size: " + str(len(mtps)))
for tp in mtps:
    print(tp)

In [ ]:
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

In [103]:
print("Size of duplicatedItems: "+ str(len(duplicatedItems)))
for tp in duplicatedItems:
    print(tp)

Size of duplicatedItems: 4
http://dbpedia.org/resource/Hells_Angels  --  http://xmlns.com/foaf/0.1/primaryTopic  --  http://en.wikipedia.org/wiki/Hells_Angels  ["Hells Angels"@en] (70.74291408061981)
http://dbpedia.org/resource/Hells_Angels  --  http://xmlns.com/foaf/0.1/isPrimaryTopicOf  --  http://en.wikipedia.org/wiki/Hells_Angels  ["Hells Angels"@en] (70.74291408061981)
http://dbpedia.org/resource/Hells_Angels  --  http://dbpedia.org/ontology/knownFor  --  http://dbpedia.org/resource/Sonny_Barger  ["Hells Angels"@en] (70.74291408061981)
http://dbpedia.org/resource/Hells_Angels  --  http://dbpedia.org/property/keyPeople  --  http://dbpedia.org/resource/Sonny_Barger  ["Hells Angels"@en] (70.74291408061981)


In [ ]:
reductionTestsDegreeOne(expandedGraph)
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

In [20]:
keepMinEdge(expandedGraph)
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Size of expanded graph: 0


# Test Purpose -- Main

In [45]:
k = np.array([['mayor', 'city']])
result = evaluate_similarity_score(wecn, k, 'xxx')
print(result[0])

0.5351437


In [46]:
w1 = 'admit'
w2 = 'admit'
print(dw(w1,w2))

1.0


In [33]:
print('1' != '2')

True


In [53]:
dw('develop','developer')

0.9555555555555555

In [ ]:
filterStringPredicates = ["wikiPageWikiLink","wikiPageRedirects","wikiPageDisambiguates", "Thing","wikiPageUsesTemplate","rdf-syntax-ns#type"]
filterStringSubjects = ["entity", "Category", "wikidata","owl#Thing", "http://wikidata.dbpedia.org/resource/Q"]
expandedGraph = []
duplicatedItems = []
line = 'Alberta, admit, province'
literalsToConnect, predicates = indentifyEntitiesAndPredicates(line.strip())
qtps = []
for literal in literalsToConnect:
    triple = Triple()
    triple.setObject(literal)
    triple.setSeeds(literal)
    qtps.append(triple)

for tp in qtps:
    print(tp.getObject())

# Test Purpose

levelOfExpansion = 1
while(levelOfExpansion < 3 and not checkConnection(literalsToConnect, duplicatedItems)):
    #add mtps to a new list
    qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
    levelOfExpansion += 1

print("Matching Triples: -----   size: " + str(len(mtps)))
for tp in mtps:
    print(tp)

In [25]:
print(isResourceInDataset("wrote"))

True


In [ ]:
queryString = 'select distinct ?s ?p where {<http://dbpedia.org/resource/The_Hunger_Games> ?p ?s}'
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["p"]["value"] + " --- " +result["s"]["value"])

In [27]:
print("jink".rindex('m'))

ValueError: substring not found